# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/neSFl/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/dsECZ/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298463787919                   -0.85    5.0         
  2   -8.300197439908       -2.76       -1.25    1.0   97.3ms
  3   -8.300432085653       -3.63       -1.89    3.0    132ms
  4   -8.300460923647       -4.54       -2.77    2.1    124ms
  5   -8.300463874533       -5.53       -3.03    3.1    146ms
  6   -8.300464243822       -6.43       -3.22   10.0    236ms
  7   -8.300464443103       -6.70       -3.38    1.8    114ms
  8   -8.300464550593       -6.97       -3.53    2.9    133ms
  9   -8.300464615033       -7.19       -3.72    4.8    189ms
 10   -8.300464630340       -7.82       -3.85    1.2    139ms
 11   -8.300464642299       -7.92       -4.14    3.2    170ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67440367394                   -0.70    7.8         
  2   -16.67867946405       -2.37       -1.14    1.8    272ms
  3   -16.67920952645       -3.28       -1.85    2.1    288ms
  4   -16.67927890927       -4.16       -2.73    2.1    284ms
  5   -16.67928600402       -5.15       -3.16    5.0    380ms
  6   -16.67928619818       -6.71       -3.45    3.8    332ms
  7   -16.67928621770       -7.71       -3.92    1.8    254ms
  8   -16.67928622145       -8.43       -4.56    2.1    278ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32608550223                   -0.56    7.0         
  2   -33.33263928956       -2.18       -1.00    1.0    928ms
  3   -33.33586960928       -2.49       -1.71    5.6    1.25s
  4   -33.33688555222       -2.99       -2.46    7.1    1.36s
  5   -33.33693400802       -4.31       -2.94    6.1    1.50s
  6   -33.33694368422       -5.01       -3.75    5.0    1.31s
  7   -33.33694377663       -7.03       -4.16    4.6    1.38s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298327737654                   -0.85    5.0         
  2   -8.300242675152       -2.72       -1.59    1.0   78.3ms
  3   -8.300407553840       -3.78       -2.42    2.5    104ms
  4   -8.300329361181   +   -4.11       -2.19    4.6    157ms
  5   -8.300464030003       -3.87       -3.49    1.0    109ms
  6   -8.300464503558       -6.32       -3.70    7.8    219ms
  7   -8.300464633918       -6.88       -4.12    1.0   92.3ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/neSFl/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32715753928                   -0.56    7.1         
  2   -33.32954208287       -2.62       -1.28    1.2    817ms
  3   -25.35375468815   +    0.90       -0.62    5.6    1.46s
  4   -33.12746498673        0.89       -1.34    4.8    1.46s
  5   -33.28515198920       -0.80       -1.42    2.6    1.09s
  6   -32.37616208890   +   -0.04       -1.07    3.9    1.20s
  7   -33.32822779249       -0.02       -2.00    3.4    1.11s
  8   -33.33380627194       -2.25       -2.26    2.9    1.09s
  9   -33.33515682625       -2.87       -2.35    2.1    988ms
 10   -33.33493146956   +   -3.65       -2.38    1.0    800ms
 11   -33.33647840733       -2.81       -2.67    2.1    923ms
 12   -33.33682311839       -3.46      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.